In [6]:
import numpy as np
import cma
import cma.purecma as purecma
import matplotlib.pyplot as plt

# Formes Canoniques d'Algorithmes Evolutionnaires

## Nomenclature générale d'un Algorithme Evolutionnaire : 
Un algorithme evolutionnaire reprend géneralement les composantes suivantes : 
* Une population d'individus (un ou plusieurs) representant des génotypes 
* Une méthode de perturbation des individus pour générer des nouveaux individus (successeurs ou offspring)
* Une fonction d'évaluation qui sert a comparer les genotypes en fonction de leur phenotypes et a en sélectionner certains.

Donc pour chaque élément les variantes les plus communes sont : 
- Pour la population : 1 Si elle se compose d'un seul élement $\mu$ si plusieurs et $\mu / \sigma$ si parmi la population un nombre $\sigma$ seulement participent a la créations des successeurs.
- Successeurs : 1 ou $\lambda$ si plusieurs successeurs.
- Mode de sélection "," si la sélection se fait parmi l'offspring ou "+" si elle se fait parmi les parents et l'offspring.

Donc par exemple : 
- (1,1) ne peut pas exister ça voudrait dire qu'on sélectionne parmi une offspring de 1 donc enfaite le nouvel élément serrait seul et on n'aurait pas avec qui le comparer on le prendrait systématiquement
- (1+1) signifie qu'on a genere un individu, qu'on perturbe a chaque étape et qu'on garde l'offspring si elle est meilleur que l'individu (donc on effectue une sélection sur le parent et l'offspring).
- (1,$\lambda$) veut dire que a chaque étape on génère plusieures descendants et on remplace le parent systématiquement par le meilleur élément de l'offspring (le parent ne participe pas a la sélection).
- ($\mu$ + $\lambda$) veut dire que la sélection de la prochaine population qui contient plusieurs individus se fait parmi plusieurs l'offspring et l'ancienne population. 

## Algorithmes evolutionnaires basiques
Dans cette partie nous allons implémenter et comparer différents algorithmes génétiques simples en les testant sur le problème du Lab 01 sur la prédiction des prix des maisons dans boston.

In [3]:
"""
On commence par l'algorithme le plus simple qu'on puisse imaginer, un (1+1) avec perturbation normale.
"""
def launch_oneplusone(center, sigma, ma_func, nbeval=10000):
    parent = np.array(center)
    i=0
    j=0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    while i<nbeval:
        child = parent + sigma*np.random.normal(0,np.ones(parent.shape),parent.shape)
        childFit = ma_func(child)
        if childFit <= parentFit:
            parentFit = childFit
            parent = np.copy(child)
            if bestFit > parentFit:
                bestFit = parentFit
                bestIt = i
                print("New best fit found : ", bestFit," a l'itteration : ",i)

        j+=1
        solutions = np.array([parent])
        i+=1
    print ("Best fit",bestFit,"at iteration",bestIt)
    return bestFit


In [4]:
"""
Dans ce qui suit nous utiliserons la structure de réseaux de neurones simple que nous avons vu précédement
car elle permet de manipuler les poids de façon "flat" c'est a dire en amount (de façon non structurée)
(Pas la peine de relire tout ça c'est pas important, c'est a titre indicatif et on aurait pu utiliser scikit-learn
remarquez juste que j'ai mis une fonction d'activation linéaire a la place de la sigmoid)
"""
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None 
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))
    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])
    
    def predict(self,x):
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            #y = sigmoid(a)
            y = a
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            """
            ICI !! 
            """
            #out = tanh(a)
            out = a
            return out
        else: # Simple monolayer perceptron
            """
            et ICI !! 
            """
            #return tanh(np.matmul(x,self.weights[0]) + self.bias[0])
            return np.matmul(x,self.weights[0]) + self.bias[0]

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
#On charge les données 
#On utilise un réseau de neurones simple 
Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
print(Network.get_parameters())

[0.9632367  0.67464808 0.695181   0.3680689  0.4550447  0.24074075
 0.74443548 0.8476449  0.44059988 0.60005399 0.01839164 0.81461847
 0.4004021  0.26640984]


In [6]:
#La fonction d'evalutation qu'on prend va associer a chaque vecteur de poids w l'erreur moyenne quadratique

def evaluation_function(w):
    X, y = load_boston(return_X_y=True)
    Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
    Network.set_parameters(w)
    y_pred = Network.predict(X)
    return mean_squared_error(y,y_pred)

Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
init_point = Network.get_parameters()
evaluation_function(init_point)


22693.000336136498

In [19]:
launch_oneplusone(init_point, 0.01, evaluation_function, nbeval=10000)

New best fit found :  31391.78567262497  a l'itteration :  2
New best fit found :  29961.907652840746  a l'itteration :  4
New best fit found :  26750.38707383033  a l'itteration :  5
New best fit found :  24920.44647672568  a l'itteration :  6
New best fit found :  21263.679289415853  a l'itteration :  10
New best fit found :  20675.413148785658  a l'itteration :  11
New best fit found :  19878.833562393815  a l'itteration :  13
New best fit found :  17934.640640893864  a l'itteration :  15
New best fit found :  16998.485576776497  a l'itteration :  20
New best fit found :  14367.764160807244  a l'itteration :  21
New best fit found :  12482.266904094566  a l'itteration :  22
New best fit found :  10821.85443067352  a l'itteration :  24
New best fit found :  10268.57180694242  a l'itteration :  25
New best fit found :  9253.042249972474  a l'itteration :  28
New best fit found :  7920.405307782994  a l'itteration :  30
New best fit found :  7410.29165825863  a l'itteration :  31
New b

New best fit found :  119.63908026047399  a l'itteration :  1159
New best fit found :  119.26200964873777  a l'itteration :  1194
New best fit found :  118.24644694553358  a l'itteration :  1220
New best fit found :  117.73518886491337  a l'itteration :  1330
New best fit found :  116.05976029865998  a l'itteration :  1350
New best fit found :  116.04311655136263  a l'itteration :  1352
New best fit found :  111.61964503762977  a l'itteration :  1355
New best fit found :  111.10340298806062  a l'itteration :  1377
New best fit found :  110.07688194925534  a l'itteration :  1388
New best fit found :  109.89118846549307  a l'itteration :  1390
New best fit found :  106.85982898144977  a l'itteration :  1424
New best fit found :  104.59716747899752  a l'itteration :  1443
New best fit found :  104.1018932616577  a l'itteration :  1517
New best fit found :  100.45626383206026  a l'itteration :  1565
New best fit found :  99.71630121972083  a l'itteration :  1566
New best fit found :  99.67

New best fit found :  47.195024446564126  a l'itteration :  8289
New best fit found :  47.074003283629786  a l'itteration :  8465
New best fit found :  46.94490539862451  a l'itteration :  8526
New best fit found :  46.80196113169656  a l'itteration :  8595
New best fit found :  46.68423952381686  a l'itteration :  8706
New best fit found :  46.61030726181373  a l'itteration :  8808
New best fit found :  46.49843799804269  a l'itteration :  8853
New best fit found :  46.392702301489834  a l'itteration :  9189
New best fit found :  46.288049083966676  a l'itteration :  9328
New best fit found :  46.270809326279775  a l'itteration :  9470
New best fit found :  46.16795778493325  a l'itteration :  9578
New best fit found :  46.12263620516568  a l'itteration :  9670
New best fit found :  46.11598929745946  a l'itteration :  9740
New best fit found :  45.82872107983396  a l'itteration :  9822
Best fit 45.82872107983396 at iteration 9822


45.82872107983396

In [21]:
"""
Petit rafinement de l'algorithme en ajoutant la règle des 1/5.
Remarque : Ce rafinement contribue a réduire la stochacité de l'algorithme et donc sont utilisation doit être
bien mesurée car elle le rend trés dépendant du point de départ
"""

def launch_ESonefifthRule(center, sigma, ma_func, nbeval=10000):
    parent = np.array(center)
    i = 0
    j = 0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    print("Parent : ", parent)
    while i < nbeval:
        child = parent + sigma * np.random.normal(0, np.ones(parent.shape), parent.shape)
        childFit = ma_func(child)
        if childFit <= parentFit:
            parentFit = childFit
            parent = np.copy(child)
            sigma = sigma*2
            if bestFit > parentFit:
                bestFit = parentFit
                bestIt = i
                print("New best fit found : ", bestFit," a l'itteration : ",i)

        else:
            sigma = sigma*(2**(-(1/4)))
        j += 1
        solutions = np.array([parent])
        i += 1
    print("Best fit", bestFit, "at iteration", bestIt)
    return bestFit


launch_ESonefifthRule(init_point, 0.1, evaluation_function, nbeval=10000)

Parent :  [0.78602377 0.93589046 0.30704964 0.42078326 0.65663521 0.36066037
 0.35953519 0.75929187 0.55131458 0.10910801 0.44257915 0.24707202
 0.27661042 0.6582214 ]
New best fit found :  30463.689170933158  a l'itteration :  0
New best fit found :  27209.30081974616  a l'itteration :  1
New best fit found :  22324.701443483973  a l'itteration :  3
New best fit found :  9801.3798819386  a l'itteration :  6
New best fit found :  9275.876058720358  a l'itteration :  21
New best fit found :  5002.745755014884  a l'itteration :  27
New best fit found :  4601.968950144203  a l'itteration :  29
New best fit found :  2917.682866846926  a l'itteration :  36
New best fit found :  1816.2279860356116  a l'itteration :  37
New best fit found :  1273.4808803599326  a l'itteration :  44
New best fit found :  713.091207313017  a l'itteration :  50
New best fit found :  673.5415547621305  a l'itteration :  60
New best fit found :  535.9300434938999  a l'itteration :  61
New best fit found :  515.010

New best fit found :  124.77363097865793  a l'itteration :  684
New best fit found :  124.74140482401506  a l'itteration :  688
New best fit found :  124.68096477126409  a l'itteration :  690
New best fit found :  124.62838290002585  a l'itteration :  691
New best fit found :  124.44062079489301  a l'itteration :  694
New best fit found :  124.23317061616224  a l'itteration :  701
New best fit found :  124.00297512409078  a l'itteration :  703
New best fit found :  123.65917573893549  a l'itteration :  707
New best fit found :  123.62344165883569  a l'itteration :  728
New best fit found :  123.61661728563128  a l'itteration :  729
New best fit found :  123.55734658598271  a l'itteration :  733
New best fit found :  123.55116854289247  a l'itteration :  739
New best fit found :  123.52649524834926  a l'itteration :  740
New best fit found :  123.45781318379854  a l'itteration :  747
New best fit found :  123.38438587336383  a l'itteration :  750
New best fit found :  123.2906562364569 

New best fit found :  111.2122063561986  a l'itteration :  1357
New best fit found :  111.14192145709717  a l'itteration :  1363
New best fit found :  110.94791408062608  a l'itteration :  1370
New best fit found :  110.82467812933342  a l'itteration :  1371
New best fit found :  110.37338823539066  a l'itteration :  1375
New best fit found :  110.23637332821481  a l'itteration :  1379
New best fit found :  109.79882044887738  a l'itteration :  1383
New best fit found :  109.74928809743282  a l'itteration :  1389
New best fit found :  109.73755141682476  a l'itteration :  1397
New best fit found :  109.73083215952533  a l'itteration :  1398
New best fit found :  109.71114706272628  a l'itteration :  1411
New best fit found :  109.56006687574941  a l'itteration :  1415
New best fit found :  109.5292829922108  a l'itteration :  1421
New best fit found :  109.49693980750708  a l'itteration :  1423
New best fit found :  109.16048923056076  a l'itteration :  1427
New best fit found :  108.7

New best fit found :  101.66285421960829  a l'itteration :  2054
New best fit found :  101.65559146421704  a l'itteration :  2059
New best fit found :  101.60188634303968  a l'itteration :  2060
New best fit found :  101.57185907245507  a l'itteration :  2063
New best fit found :  101.53734478817393  a l'itteration :  2068
New best fit found :  101.48438561211991  a l'itteration :  2072
New best fit found :  101.45306740751684  a l'itteration :  2074
New best fit found :  101.42862303971904  a l'itteration :  2075
New best fit found :  101.33654800306294  a l'itteration :  2077
New best fit found :  101.14822434370889  a l'itteration :  2084
New best fit found :  100.96137976858198  a l'itteration :  2085
New best fit found :  100.91561572631093  a l'itteration :  2090
New best fit found :  100.78389658158589  a l'itteration :  2097
New best fit found :  100.6836547653349  a l'itteration :  2102
New best fit found :  99.99582636077008  a l'itteration :  2107
New best fit found :  99.88

New best fit found :  93.24081090403457  a l'itteration :  2698
New best fit found :  93.23609988349388  a l'itteration :  2704
New best fit found :  93.22425228858327  a l'itteration :  2708
New best fit found :  93.20449704864957  a l'itteration :  2713
New best fit found :  93.18787180010064  a l'itteration :  2714
New best fit found :  93.16547808779927  a l'itteration :  2720
New best fit found :  93.15689469345688  a l'itteration :  2723
New best fit found :  93.11186267525969  a l'itteration :  2724
New best fit found :  93.01380242068473  a l'itteration :  2725
New best fit found :  92.95435079717696  a l'itteration :  2734
New best fit found :  92.85313878098653  a l'itteration :  2738
New best fit found :  92.46774083924616  a l'itteration :  2740
New best fit found :  92.20794455842771  a l'itteration :  2745
New best fit found :  92.17676799478923  a l'itteration :  2748
New best fit found :  92.01197883351306  a l'itteration :  2755
New best fit found :  91.93161767263153 

...

New best fit found :  60.85423929187017  a l'itteration :  7976
New best fit found :  60.85176929902583  a l'itteration :  7981
New best fit found :  60.84546560712583  a l'itteration :  7985
New best fit found :  60.84029244641619  a l'itteration :  7991
New best fit found :  60.83894242204068  a l'itteration :  7993
New best fit found :  60.83831915582724  a l'itteration :  8000
New best fit found :  60.83617675657441  a l'itteration :  8011
New best fit found :  60.82720603682765  a l'itteration :  8012
New best fit found :  60.820558196569294  a l'itteration :  8013
New best fit found :  60.8180079681828  a l'itteration :  8021
New best fit found :  60.79311127179263  a l'itteration :  8022
New best fit found :  60.77522617023171  a l'itteration :  8030
New best fit found :  60.76855232976226  a l'itteration :  8035
New best fit found :  60.768030246303354  a l'itteration :  8047
New best fit found :  60.76509256360843  a l'itteration :  8048
New best fit found :  60.76432410964629

New best fit found :  59.5168193837595  a l'itteration :  8633
New best fit found :  59.509912777347466  a l'itteration :  8639
New best fit found :  59.50347416418333  a l'itteration :  8642
New best fit found :  59.50032825904995  a l'itteration :  8646
New best fit found :  59.49240108898796  a l'itteration :  8649
New best fit found :  59.47610083273415  a l'itteration :  8655
New best fit found :  59.474673584321316  a l'itteration :  8659
New best fit found :  59.4706403685704  a l'itteration :  8665
New best fit found :  59.46855875362843  a l'itteration :  8676
New best fit found :  59.46792107782435  a l'itteration :  8679
New best fit found :  59.46318738761589  a l'itteration :  8680
New best fit found :  59.462571228046606  a l'itteration :  8681
New best fit found :  59.454368803788576  a l'itteration :  8684
New best fit found :  59.435566407010704  a l'itteration :  8689
New best fit found :  59.38274214386007  a l'itteration :  8694
New best fit found :  59.378310344067

New best fit found :  57.75079929227875  a l'itteration :  9309
New best fit found :  57.74979263851092  a l'itteration :  9318
New best fit found :  57.74829717830486  a l'itteration :  9319
New best fit found :  57.74670240862572  a l'itteration :  9321
New best fit found :  57.74640814432888  a l'itteration :  9328
New best fit found :  57.74040070640662  a l'itteration :  9336
New best fit found :  57.74037670351518  a l'itteration :  9337
New best fit found :  57.73885791023329  a l'itteration :  9346
New best fit found :  57.73582346027885  a l'itteration :  9347
New best fit found :  57.73177506393892  a l'itteration :  9349
New best fit found :  57.73041107256125  a l'itteration :  9358
New best fit found :  57.721234300178835  a l'itteration :  9362
New best fit found :  57.7179196519259  a l'itteration :  9365
New best fit found :  57.71049010757973  a l'itteration :  9369
New best fit found :  57.70155039029523  a l'itteration :  9376
New best fit found :  57.689300665058205

New best fit found :  56.463883836379786  a l'itteration :  9945
New best fit found :  56.46058915348272  a l'itteration :  9954
New best fit found :  56.45376244003095  a l'itteration :  9957
New best fit found :  56.44933932508427  a l'itteration :  9961
New best fit found :  56.44705740053829  a l'itteration :  9962
New best fit found :  56.415467747375274  a l'itteration :  9971
New best fit found :  56.40569027074711  a l'itteration :  9976
New best fit found :  56.38710435761823  a l'itteration :  9977
New best fit found :  56.38285126554366  a l'itteration :  9983
New best fit found :  56.37825680918094  a l'itteration :  9989
New best fit found :  56.36634680405849  a l'itteration :  9995
Best fit 56.36634680405849 at iteration 9995


56.36634680405849

In [25]:
def launch_LambdaES(center, _lambda, sigma,ma_func, nbeval=10000):
    center = np.array(center)
    parent = center + sigma * np.random.normal(0, center.shape, center.shape[0])
    i = 0
    j = 0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    print("Parent : ", parent)
    while i < nbeval:
        for k in range(_lambda):
            child = parent + sigma * np.random.normal(0, np.ones(parent.shape), parent.shape)
            childFit = ma_func(child)
            if childFit <= parentFit:
                parentFit = childFit
                parent = np.copy(child)
                sigma = sigma * 2
                if bestFit > parentFit:
                    bestFit = parentFit
                    bestIt = i
                    print("New best fit found : ", bestFit," a l'itteration : ",i)
            else:
                sigma = sigma * (2 ** (-(1 / 4)))
            j += 1
            solutions = np.array([parent])
        i += 1
    print("Best fit", bestFit, "at iteration", bestIt)
    return bestFit

launch_LambdaES(init_point,100, 0.01, evaluation_function, nbeval=1000)

Parent :  [0.64056104 0.92043333 0.38542168 0.34078264 0.72102249 0.35228642
 0.37505134 0.96229866 0.44654743 0.26673939 0.45941968 0.31192804
 0.323009   0.59042412]
New best fit found :  70452.63176938653  a l'itteration :  0
New best fit found :  68319.83406669935  a l'itteration :  0
New best fit found :  59665.294902154026  a l'itteration :  0
New best fit found :  57914.45169678753  a l'itteration :  0
New best fit found :  37876.52933672275  a l'itteration :  0
New best fit found :  22555.262274898185  a l'itteration :  0
New best fit found :  15406.89299980219  a l'itteration :  0
New best fit found :  2409.4786774193126  a l'itteration :  0
New best fit found :  833.2960186901497  a l'itteration :  0
New best fit found :  812.5449077835136  a l'itteration :  0
New best fit found :  778.1911984207596  a l'itteration :  0
New best fit found :  553.1913050674976  a l'itteration :  0
New best fit found :  494.01671087816567  a l'itteration :  0
New best fit found :  472.010140815

New best fit found :  212.67065839285962  a l'itteration :  6
New best fit found :  212.01443589141553  a l'itteration :  6
New best fit found :  210.93169666110074  a l'itteration :  6
New best fit found :  210.90995250093027  a l'itteration :  6
New best fit found :  210.8479417448721  a l'itteration :  6
New best fit found :  210.791178173494  a l'itteration :  6
New best fit found :  210.52175879359964  a l'itteration :  7
New best fit found :  210.07058209455957  a l'itteration :  7
New best fit found :  209.68715941012337  a l'itteration :  7
New best fit found :  209.57781009679104  a l'itteration :  7
New best fit found :  209.48412838476946  a l'itteration :  7
New best fit found :  208.79041995963505  a l'itteration :  7
New best fit found :  208.37547260277006  a l'itteration :  7
New best fit found :  207.619032928912  a l'itteration :  7
New best fit found :  207.51523927892066  a l'itteration :  7
New best fit found :  207.2253062504496  a l'itteration :  7
New best fit f

New best fit found :  153.21097183630127  a l'itteration :  13
New best fit found :  152.7631341877043  a l'itteration :  13
New best fit found :  152.57775554213032  a l'itteration :  13
New best fit found :  152.5362583835314  a l'itteration :  13
New best fit found :  151.91557381513888  a l'itteration :  13
New best fit found :  151.36726099887295  a l'itteration :  13
New best fit found :  151.28901527675006  a l'itteration :  13
New best fit found :  150.45443120990606  a l'itteration :  13
New best fit found :  148.81513509509796  a l'itteration :  13
New best fit found :  147.5126340484222  a l'itteration :  13
New best fit found :  144.56940393393873  a l'itteration :  13
New best fit found :  144.52683695534918  a l'itteration :  14
New best fit found :  144.2111204614015  a l'itteration :  14
New best fit found :  143.4289912112209  a l'itteration :  14
New best fit found :  142.3909412208279  a l'itteration :  14
New best fit found :  141.46970502645473  a l'itteration :  1

New best fit found :  119.6950884447188  a l'itteration :  20
New best fit found :  119.68248661648816  a l'itteration :  20
New best fit found :  119.6795808680258  a l'itteration :  20
New best fit found :  119.67950478930803  a l'itteration :  20
New best fit found :  119.67854069332388  a l'itteration :  20
New best fit found :  119.63818517918754  a l'itteration :  20
New best fit found :  119.45583220877137  a l'itteration :  20
New best fit found :  119.42848989711237  a l'itteration :  20
New best fit found :  119.41383743269246  a l'itteration :  20
New best fit found :  119.40444899087056  a l'itteration :  20
New best fit found :  119.23548653251595  a l'itteration :  20
New best fit found :  119.12312065938181  a l'itteration :  20
New best fit found :  119.08423668664865  a l'itteration :  20
New best fit found :  119.06827027380209  a l'itteration :  20
New best fit found :  119.04515097796238  a l'itteration :  21
New best fit found :  118.82421590223014  a l'itteration 

New best fit found :  100.60793416807074  a l'itteration :  27
New best fit found :  100.57723579338263  a l'itteration :  27
New best fit found :  100.41278302256895  a l'itteration :  27
New best fit found :  100.40836714755473  a l'itteration :  27
New best fit found :  100.37213786039727  a l'itteration :  27
New best fit found :  100.35337426160825  a l'itteration :  27
New best fit found :  100.26606139260767  a l'itteration :  27
New best fit found :  100.26203914727793  a l'itteration :  27
New best fit found :  100.24209876076193  a l'itteration :  27
New best fit found :  100.18849257003157  a l'itteration :  27
New best fit found :  100.03689820326329  a l'itteration :  27
New best fit found :  100.00448503590914  a l'itteration :  27
New best fit found :  99.90581444357797  a l'itteration :  27
New best fit found :  99.88064098585026  a l'itteration :  27
New best fit found :  99.79334722442222  a l'itteration :  27
New best fit found :  99.761143975216  a l'itteration :  2

...

New best fit found :  43.50095076209323  a l'itteration :  384
New best fit found :  43.50088555264425  a l'itteration :  385
New best fit found :  43.500873364034106  a l'itteration :  385
New best fit found :  43.50083312699084  a l'itteration :  385
New best fit found :  43.50082740258791  a l'itteration :  385
New best fit found :  43.500345862213656  a l'itteration :  385
New best fit found :  43.50024941618405  a l'itteration :  385
New best fit found :  43.500101609540394  a l'itteration :  385
New best fit found :  43.50007491777288  a l'itteration :  385
New best fit found :  43.49971614210379  a l'itteration :  385
New best fit found :  43.49965192318573  a l'itteration :  385
New best fit found :  43.499255414303484  a l'itteration :  385
New best fit found :  43.49894146773177  a l'itteration :  385
New best fit found :  43.498098959880075  a l'itteration :  385
New best fit found :  43.49789447133349  a l'itteration :  385
New best fit found :  43.49582309847243  a l'itter

New best fit found :  43.457098395657184  a l'itteration :  391
New best fit found :  43.45687129018385  a l'itteration :  391
New best fit found :  43.45537241146593  a l'itteration :  391
New best fit found :  43.455040804824804  a l'itteration :  391
New best fit found :  43.454732254172306  a l'itteration :  391
New best fit found :  43.45432645735486  a l'itteration :  391
New best fit found :  43.45416787485327  a l'itteration :  392
New best fit found :  43.45403697270662  a l'itteration :  392
New best fit found :  43.454029241346234  a l'itteration :  392
New best fit found :  43.4539349598518  a l'itteration :  392
New best fit found :  43.45375216914105  a l'itteration :  392
New best fit found :  43.45327201459549  a l'itteration :  392
New best fit found :  43.45241739790743  a l'itteration :  392
New best fit found :  43.4519832342487  a l'itteration :  392
New best fit found :  43.45173134409625  a l'itteration :  392
New best fit found :  43.45149739889812  a l'itterati

New best fit found :  43.39825708000245  a l'itteration :  398
New best fit found :  43.39814716696892  a l'itteration :  398
New best fit found :  43.39786074486948  a l'itteration :  398
New best fit found :  43.39708428597373  a l'itteration :  398
New best fit found :  43.396727266581614  a l'itteration :  398
New best fit found :  43.39506577931001  a l'itteration :  398
New best fit found :  43.394854508118215  a l'itteration :  399
New best fit found :  43.39455836535577  a l'itteration :  399
New best fit found :  43.39441767079263  a l'itteration :  399
New best fit found :  43.39440483520964  a l'itteration :  399
New best fit found :  43.39432779906128  a l'itteration :  399
New best fit found :  43.39390964901219  a l'itteration :  399
New best fit found :  43.39245199567692  a l'itteration :  399
New best fit found :  43.39228721739438  a l'itteration :  399
New best fit found :  43.39115707645015  a l'itteration :  399
New best fit found :  43.39114136012738  a l'itterati

New best fit found :  43.31108467514586  a l'itteration :  405
New best fit found :  43.31087089006397  a l'itteration :  405
New best fit found :  43.31079283243567  a l'itteration :  405
New best fit found :  43.31068626183796  a l'itteration :  405
New best fit found :  43.31063279577779  a l'itteration :  405
New best fit found :  43.31060212421397  a l'itteration :  406
New best fit found :  43.31053987684707  a l'itteration :  406
New best fit found :  43.31041319618385  a l'itteration :  406
New best fit found :  43.31039738026606  a l'itteration :  406
New best fit found :  43.310187001591345  a l'itteration :  406
New best fit found :  43.310161218916406  a l'itteration :  406
New best fit found :  43.3099155968879  a l'itteration :  406
New best fit found :  43.30966575852958  a l'itteration :  406
New best fit found :  43.30946932458984  a l'itteration :  406
New best fit found :  43.30920779968647  a l'itteration :  406
New best fit found :  43.30915406794987  a l'itteratio

KeyboardInterrupt: 

In [31]:
res = cma.CMAEvolutionStrategy(init_point, 0.01).optimize(evaluation_function,maxfun=10000).result


(5_w,11)-aCMA-ES (mu_w=3.4,w_1=42%) in dimension 14 (seed=186472, Sun Apr  5 17:20:03 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     11 2.677748941043584e+04 1.0e+00 9.97e-03  1e-02  1e-02 0:00.0
    2     22 2.430381408264909e+04 1.2e+00 1.05e-02  1e-02  1e-02 0:00.1
    3     33 2.242092945093028e+04 1.3e+00 1.13e-02  1e-02  1e-02 0:00.1
   90    990 1.303434946956036e+02 1.1e+01 4.93e-02  2e-02  8e-02 0:03.2
  100   1100 1.118616503130179e+02 1.3e+01 4.94e-02  2e-02  7e-02 0:03.5
  200   2200 4.051302124357185e+01 9.0e+01 6.71e-02  7e-03  2e-01 0:07.0
  300   3300 2.448377854116623e+01 3.8e+02 1.20e-02  6e-04  4e-02 0:10.5
  400   4400 2.421149300803299e+01 1.5e+03 1.79e-02  5e-04  9e-02 0:13.9
  500   5500 2.388757008023257e+01 3.1e+03 6.68e-02  1e-03  4e-01 0:17.5
  600   6600 2.269243868059143e+01 1.0e+04 1.32e-01  1e-03  2e+00 0:21.0
  700   7700 2.189492637069410e+01 1.6e+04 6.62e-03  4e-05  6e-02 0:24.4
  800   8800 2.189483118222963e+0

In [33]:
res = purecma.fmin(evaluation_function,init_point,0.01,maxfevals=10000)

evals: ax-ratio max(std)   f-value
   11:     1.0  1.0e-02  26756.298552985405
   22:     1.0  1.1e-02  22243.83707573546
 1100:     3.5  3.7e-02  123.58215609713875
 2200:    11.7  6.0e-02  70.62960484053689
 3300:    27.0  2.8e-02  48.91834856633729
 4400:    85.1  2.9e-02  40.84695856654236
 5500:   350.7  1.9e-01  26.925254877824067
 6600:   507.1  1.5e-01  25.401432495121423
 7700:   652.1  1.6e-02  24.795125474489517
 8800:  2748.1  2.8e-01  24.130824999270143
 9900:  3876.5  1.7e+00  23.678116852441107
evals: ax-ratio max(std)   f-value
10010:  4189.4  1.1e+00  23.440399664127938
termination by {'maxfevals': 10000}
best f-value = 23.44021257070262
solution = [-0.08338652378623565, 0.03333296703714241, -0.09626104762308149, 3.4306421702739254, 1.8575690265576628, 4.66154686705747, -0.012465332686858198, -1.0447720195527097, 0.1806512315210863, -0.010705964213907435, -0.5437209875141128, 0.009398929345407566, -0.5135862071394197, 13.70115517231868]
